# Collision Avoidance - Train Model（避障-训练模型）
欢迎来到Host（PC）端的Jupyter！如果你浏览一下小车上运行的笔记本，看起来应该很熟悉。在这个笔记本中，我们将训练图像分类器来检测``free `` 和``blocked`` 两个类 ，用于避撞。在此使用一个流行的 *Pytorch* 深度学习库

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### 上传和提取数据集
在开始之前，应该从小车的 ``data-collection.ipynb`` 笔记本中将创建的“dataset.zip”文件上传到这里，然后就可以调用下面的命令来提取这个数据集

In [ ]:
!unzip -q dataset.zip

您应该会在文件浏览器中看到一个名为 ``dataset`` 的文件夹。

### 创建数据集实例
现在我们使用 ``torchvision.datasets`` 包提供的 ``ImageFolder`` 数据集类。我们附加 ``torchvision.transforms`` 包中的转换，以准备训练数据。

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### 将数据集拆分为训练集和测试集
接下来，我们将数据集分成 *training* 和 *test* 两组。测试组将用于验证我们训练的模型的准确性。

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

### 创建数据加载程序以批量加载数据
我们将创建两个 ``DataLoader`` 实例，这些实例为数据洗牌、生成 *batch* 图像和与多个工作线程并行加载示例提供实用程序

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### 定义神经网络

现在，我们定义要训练的神经网络。*torchvision* 软件包提供了一组我们可以使用的预先训练的模型。

在一个名为 *迁移学习（transfer learning）* 的过程中，我们可以重新利用一个预先训练好的模型（在数百万张图片上进行训练）来完成一个新的任务，这个任务的可用数据可能要少得多。
在最初训练的预训练模型中学习到的重要特征可用于新任务。我们将使用 ``alexnet`` 模型。

In [ ]:
model = models.alexnet(pretrained=True)

``alexnet`` 模型最初是为有1000个类标签的数据集训练的，但是我们的数据集只有两个类标签！我们会将最后一层替换成个新的/未经训练的层，只有两个输出。

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

最后，我们将模型转移到GPU上执行

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### 训练神经网络

使用下面的代码，我们将神经网络训练 30 个 epoch，在每个阶段之后保存性能最佳的模型。
> 每个 epoch 是我们数据的完整记录。

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

完成后，您应该在Jupyter Lab文件浏览器中看到一个文件“best_model.pth”。选择“右键单击”将模型下载到工作站